In [19]:
import urllib3
import json
import requests
import io
import pandas as pd
import zipp
import os
import numpy as np
import geopandas as gpd
from shapely.geometry import LineString
import matplotlib.pyplot as plt
import contextily as ctx
from matplotlib_scalebar.scalebar import ScaleBar

In [20]:
def extract_GTFS(date): #write as 'yyyy-mm-dd'
    print (f'extracting GTFS for {date}')

    year, month, day=date.split('-')
    path=f'https://openbus-stride-public.s3.eu-west-1.amazonaws.com/gtfs_archive/{year}/{month}/{day}/israel-public-transportation.zip'
    response = requests.get(path)
    response.raise_for_status()  # Ensure the request was successful
    archive = zipp.Path(io.BytesIO(response.content))

    dfs = {}
    print("Files in the archive:", list(archive.iterdir()))

    for file_path in archive.iterdir():
        if file_path.name.endswith('.txt'):  # Check for GTFS .txt files
            # Read the file's content into a Pandas DataFrame
            content = file_path.read_text(encoding='utf-8')  # Read the file as text
            dfs[file_path.name] = pd.read_csv(io.StringIO(content))
            print(f"Loaded {file_path.name} into a DataFrame")

    return dfs

    # Example: Access a specific DataFrame
    # print(dfs["stops.txt"].head())  # Replace "stops.txt" with the desired file name

In [21]:
def create_lines_gdf(GTFS):

    # Group by shape_id and sort by sequence
    shapes_grouped = GTFS['shapes.txt'].sort_values(by=["shape_id", "shape_pt_sequence"]).groupby("shape_id")

    # Create a list of LineStrings for each shape_id
    shape_lines = []
    for shape_id, group in shapes_grouped:
        points = list(zip(group["shape_pt_lon"], group["shape_pt_lat"]))
        line = LineString(points)
        shape_lines.append({"shape_id": shape_id, "geometry": line})

    # Create GeoDataFrame
    gdf = gpd.GeoDataFrame(shape_lines, crs="EPSG:4326")

    shapes=gdf.merge(GTFS['trips.txt'][['route_id', 'shape_id']].drop_duplicates())
    shapes=shapes.merge(GTFS['routes.txt'], on='route_id', how='left')
    
    return shapes

In [22]:
def create_lines_gdf(GTFS):

    # Group by shape_id and sort by sequence
    shapes_grouped = GTFS['shapes.txt'].sort_values(by=["shape_id", "shape_pt_sequence"]).groupby("shape_id")

    # Create a list of LineStrings for each shape_id
    shape_lines = []
    for shape_id, group in shapes_grouped:
        points = list(zip(group["shape_pt_lon"], group["shape_pt_lat"]))
        line = LineString(points)
        shape_lines.append({"shape_id": shape_id, "geometry": line})

    return shape_lines

In [23]:
def create_map(shapes, cmap):
    gdf_web = shapes.to_crs(epsg=3857)

    # Plot
    fig, ax = plt.subplots(figsize=(10, 10))
    gdf_web.plot(ax=ax, linewidth=2, column="route_short_name",
                 legend=True, cmap=cmap)

    # Add grey basemap
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)

    # Add scale bar
    scalebar = ScaleBar(1, units="m", location='lower right')  # 1 pixel = 1 meter in EPSG:3857
    ax.add_artist(scalebar)

    # Add north arrow
    arrow = mpatches.FancyArrowPatch((0.95, 0.1), (0.95, 0.2),
                                    transform=ax.transAxes,
                                    arrowstyle='simple', color='black')
    ax.add_patch(arrow)
    ax.text(0.95, 0.21, 'N', transform=ax.transAxes,
            horizontalalignment='center', fontsize=12)

    # Add grid (graticule lines)
    ax.grid(True, which='both', linestyle='--', color='lightgrey', alpha=0.6)

    # Remove axes
    ax.set_axis_off()

    plt.tight_layout()
    
    plt.show()


In [24]:
def create_year(date):
    GTFS=extract_GTFS(date)
    shape_lines=create_lines_gdf(GTFS)       
    gdf = gpd.GeoDataFrame(shape_lines, crs="EPSG:4326")
    shapes=gdf.merge(GTFS['trips.txt'][['route_id', 'shape_id']].drop_duplicates())
    shapes=shapes.merge(GTFS['routes.txt'], on='route_id', how='left')
    shapes[['officelineid', 'direction', 'alternative']]=shapes['route_desc'].str.split('-', expand=True)
    return shapes.drop(columns=
                      ['shape_id','route_id','agency_id','route_long_name','route_type','route_color'])

In [25]:
#extract lines by year
L_2025=create_year('2025-04-01')
L_2024=create_year('2024-04-01')

extracting GTFS for 2025-04-01
Files in the archive: [Path(None, 'agency.txt'), Path(None, 'calendar.txt'), Path(None, 'fare_attributes.txt'), Path(None, 'fare_rules.txt'), Path(None, 'routes.txt'), Path(None, 'shapes.txt'), Path(None, 'stop_times.txt'), Path(None, 'stops.txt'), Path(None, 'translations.txt'), Path(None, 'trips.txt')]
Loaded agency.txt into a DataFrame
Loaded calendar.txt into a DataFrame
Loaded fare_attributes.txt into a DataFrame
Loaded fare_rules.txt into a DataFrame
Loaded routes.txt into a DataFrame
Loaded shapes.txt into a DataFrame
Loaded stop_times.txt into a DataFrame
Loaded stops.txt into a DataFrame
Loaded translations.txt into a DataFrame
Loaded trips.txt into a DataFrame
extracting GTFS for 2024-04-01
Files in the archive: [Path(None, 'agency.txt'), Path(None, 'calendar.txt'), Path(None, 'fare_attributes.txt'), Path(None, 'fare_rules.txt'), Path(None, 'routes.txt'), Path(None, 'shapes.txt'), Path(None, 'stop_times.txt'), Path(None, 'stops.txt'), Path(None,

In [26]:
# select required lines by year
shapes_to_map_2025=L_2025[L_2025['officelineid'].isin(['31033', '35034'])&(L_2025['route_short_name'].isin(['33', '34']))].copy()
shapes_to_map_2024=L_2024[(L_2024['officelineid'].isin(['47002', '28013']))].copy()

In [27]:
shapes_to_map_2024=L_2024[(L_2024['route_desc'].isin(['15019-2-#']))].copy()

shapes_to_map_2024

,geometry,route_short_name,route_desc,officelineid,direction,alternative
999,"LINESTRING (35.24575 31.79010, 35.24550 31.790...",19,15019-2-#,15019,2,#
